In [77]:
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np
from nltk.metrics import ConfusionMatrix

In [69]:
s1_train = pd.read_csv('Dataset/s1_train.csv', sep=",", header=None,index_col = False)
s1_test = pd.read_csv('Dataset/s1_test.csv', sep=",", header=None,index_col = False)
s2_train = pd.read_csv('Dataset/s2_train.csv', sep=",", header=None,index_col = False)
s2_test = pd.read_csv('Dataset/s2_test.csv', sep=",", header=None,index_col = False)

In [12]:
#get the list of words
whole_ds = s1_train.append(s1_test,ignore_index=True )
whole_ds['tokenized_sents'] = whole_ds.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
print(whole_ds.shape)

all_words = []

for line in whole_ds['tokenized_sents']:
    for w in line:
        all_words.append(w.lower())
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

(285, 3)


In [29]:
#convert sentences to bag of words feature vector
def convert_features(sentence):
    return {i:(i in sentence) for i in word_features}

In [70]:
#stage 1 classification
s1_train['tokenized_sents'] = s1_train.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
s1_featured_train = [(convert_features(row['tokenized_sents']), row[1]) for index, row in s1_train.iterrows()]
s1_test['tokenized_sents'] = s1_test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
s1_featured_test = [(convert_features(row['tokenized_sents']), row[1]) for index, row in s1_test.iterrows()]

classifier_s1= nltk.NaiveBayesClassifier.train(s1_featured_train)
print(nltk.classify.accuracy(classifier_s1, s1_featured_test))

0.9154929577464789


In [33]:
'''gnb = GaussianNB()
y_pred = gnb.fit(featuresets[:200][0],featuresets[:200][1]).predict(featuresets[201:][0])
'''

'gnb = GaussianNB()\ny_pred = gnb.fit(featuresets[:200][0],featuresets[:200][1]).predict(featuresets[201:][0])\n'

In [74]:
#stage 2
s2_train['tokenized_sents'] = s2_train.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
s2_featured_train = [(convert_features(row['tokenized_sents']), row[1]) for index, row in s2_train.iterrows()]
s2_test['tokenized_sents'] = s2_test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
s2_featured_test = [(convert_features(row['tokenized_sents']), row[1]) for index, row in s2_test.iterrows()]

classifier_s2 = nltk.NaiveBayesClassifier.train(s2_featured_train)
print(nltk.classify.accuracy(classifier_s2, s2_featured_test))

0.6698113207547169


In [75]:
test = pd.read_csv('Dataset/test.csv', sep=",", header=None,index_col = False)
test['tokenized_sents'] = test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

pred = np.zeros((test.shape[0],1))

for index, row in test.iterrows():
    featured_item = (convert_features(row['tokenized_sents']))
    s1_pred = classifier_s1.classify(featured_item)
    if s1_pred == 0:
        pred[index] = 0
    else:
        s2_pred = classifier_s2.classify(featured_item)
        pred[index] = s2_pred
        
        
test['prediction'] = pred

In [78]:
confusion_matrix = np.zeros((4,4))
for index, row in test.iterrows():
    ind_i = row[1]
    ind_j = int(row['prediction'])
    confusion_matrix[ind_i][ind_j] = confusion_matrix[ind_i][ind_j] + 1
        
print(confusion_matrix)

accuracy = (confusion_matrix[1,1]+confusion_matrix[2,2]+confusion_matrix[0,0]+confusion_matrix[3,3])/test.shape[0]
print(accuracy)

[[35.  0.  0.  2.]
 [ 3. 20.  3.  3.]
 [ 3.  8. 28.  6.]
 [ 7.  4.  4. 17.]]
0.6993006993006993
